### Find Word Group Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [8]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 6/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [9]:
## Master File
#path_folder_file = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 Master File.xlsx"
#sheet = "Sheet6"  # Sheet1
#file_ext = "Sixgram"

In [10]:
ngram = "Eightgram"
path_folder_file = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/\
Result/1-Select Twogram Threegram According To Word Usage Limit/{lang_folder.capitalize()}_{ngram}_Selected_With_{word_end}_Word.xlsx"
sheet = "Sheet1"
file_ext = ngram

In [11]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
495,bence senden daha iyi bir özel ajan oldum,35
496,şeyi olduğu gibi bırakın ve bırakın sözleriniz...,35
497,piyasada arabaları çabucak en yüksek fiyata el...,35
498,dostum işinizin ehli olduğunuzu biliyor ve say...,35


In [12]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

500

In [13]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [14]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [15]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [16]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [17]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [18]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,bir iki üç dört beş altı yedi sekiz,283.680,293.190,tek bir iki üç dört beş altı yedi sekiz dokuz ...,XDbW2HYrxSA
1,bir iki üç dört beş altı yedi sekiz,372.100,376.900,bir iki üç dört beş altı yedi sekiz,Y5G56jyilHQ
2,bir iki üç dört beş altı yedi sekiz,1781.400,1794.990,bak bir iki üç dört beş altı yedi sekiz dokuz ...,_wkFZF0ZuMU
3,bir iki üç dört beş altı yedi sekiz,1028.281,1034.356,bir iki üç dört beş altı yedi sekiz dokuz on,tQRsEpIj5uo
4,bir iki üç dört beş altı yedi sekiz,3326.586,3333.226,bir iki üç dört beş altı yedi sekiz dokuz on,psNiD0Ks_uk
...,...,...,...,...,...
73,hey hey hey hey hey hey hey hey,6455.185,6457.661,hey hey hey hey hey hey hey hey hey hey,_IimozO_x50
74,bu iyi bir şey mi kötü bir şey,4376.620,4378.460,acaba bu iyi bir şey mi kötü bir şey mi,KrLItbrSnZs
75,bu iyi bir şey mi kötü bir şey,4366.920,4368.320,bu iyi bir şey mi kötü bir şey mi,1mQL15DyOcM
76,dur dur dur dur dur dur dur dur,372.480,379.290,dur da ölmek çok gelecek şu anda biliyor musun...,OXmeI2243Ws


In [19]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bir iki üç dört beş altı yedi sekiz,284.023735,288.263133,tek bir iki üç dört beş altı yedi sekiz dokuz ...,XDbW2HYrxSA
1,bir iki üç dört beş altı yedi sekiz,372.100000,376.900000,bir iki üç dört beş altı yedi sekiz,Y5G56jyilHQ
2,bir iki üç dört beş altı yedi sekiz,1781.833723,1787.182979,bak bir iki üç dört beş altı yedi sekiz dokuz ...,_wkFZF0ZuMU
3,bir iki üç dört beş altı yedi sekiz,1028.281000,1033.251455,bir iki üç dört beş altı yedi sekiz dokuz on,tQRsEpIj5uo
4,bir iki üç dört beş altı yedi sekiz,3326.586000,3332.018727,bir iki üç dört beş altı yedi sekiz dokuz on,psNiD0Ks_uk
...,...,...,...,...,...
73,hey hey hey hey hey hey hey hey,6455.185000,6457.216590,hey hey hey hey hey hey hey hey hey hey,_IimozO_x50
74,bu iyi bir şey mi kötü bir şey,4376.855897,4378.365641,acaba bu iyi bir şey mi kötü bir şey mi,KrLItbrSnZs
75,bu iyi bir şey mi kötü bir şey,4366.920000,4368.235152,bu iyi bir şey mi kötü bir şey mi,1mQL15DyOcM
76,dur dur dur dur dur dur dur dur,375.612600,377.859900,dur da ölmek çok gelecek şu anda biliyor musun...,OXmeI2243Ws


In [20]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bir iki üç dört beş altı yedi sekiz,283.423735,288.863133,tek bir iki üç dört beş altı yedi sekiz dokuz ...,XDbW2HYrxSA
1,bir iki üç dört beş altı yedi sekiz,371.500000,377.500000,bir iki üç dört beş altı yedi sekiz,Y5G56jyilHQ
2,bir iki üç dört beş altı yedi sekiz,1781.233723,1787.782979,bak bir iki üç dört beş altı yedi sekiz dokuz ...,_wkFZF0ZuMU
3,bir iki üç dört beş altı yedi sekiz,1027.681000,1033.851455,bir iki üç dört beş altı yedi sekiz dokuz on,tQRsEpIj5uo
4,bir iki üç dört beş altı yedi sekiz,3325.986000,3332.618727,bir iki üç dört beş altı yedi sekiz dokuz on,psNiD0Ks_uk
...,...,...,...,...,...
73,hey hey hey hey hey hey hey hey,6454.585000,6457.816590,hey hey hey hey hey hey hey hey hey hey,_IimozO_x50
74,bu iyi bir şey mi kötü bir şey,4376.255897,4378.965641,acaba bu iyi bir şey mi kötü bir şey mi,KrLItbrSnZs
75,bu iyi bir şey mi kötü bir şey,4366.320000,4368.835152,bu iyi bir şey mi kötü bir şey mi,1mQL15DyOcM
76,dur dur dur dur dur dur dur dur,375.012600,378.459900,dur da ölmek çok gelecek şu anda biliyor musun...,OXmeI2243Ws


In [21]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,bir iki üç dört beş altı yedi sekiz,283,289,tek bir iki üç dört beş altı yedi sekiz dokuz ...,XDbW2HYrxSA
1,bir iki üç dört beş altı yedi sekiz,372,378,bir iki üç dört beş altı yedi sekiz,Y5G56jyilHQ
2,bir iki üç dört beş altı yedi sekiz,1781,1788,bak bir iki üç dört beş altı yedi sekiz dokuz ...,_wkFZF0ZuMU
3,bir iki üç dört beş altı yedi sekiz,1028,1034,bir iki üç dört beş altı yedi sekiz dokuz on,tQRsEpIj5uo
4,bir iki üç dört beş altı yedi sekiz,3326,3333,bir iki üç dört beş altı yedi sekiz dokuz on,psNiD0Ks_uk
...,...,...,...,...,...
73,hey hey hey hey hey hey hey hey,6455,6458,hey hey hey hey hey hey hey hey hey hey,_IimozO_x50
74,bu iyi bir şey mi kötü bir şey,4376,4379,acaba bu iyi bir şey mi kötü bir şey mi,KrLItbrSnZs
75,bu iyi bir şey mi kötü bir şey,4366,4369,bu iyi bir şey mi kötü bir şey mi,1mQL15DyOcM
76,dur dur dur dur dur dur dur dur,375,378,dur da ölmek çok gelecek şu anda biliyor musun...,OXmeI2243Ws


In [22]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,bir iki üç dört beş altı yedi sekiz,283,289,tek bir iki üç dört beş altı yedi sekiz dokuz ...,XDbW2HYrxSA,https://www.youtube.com/watch?v=XDbW2HYrxSA&t=...
1,bir iki üç dört beş altı yedi sekiz,372,378,bir iki üç dört beş altı yedi sekiz,Y5G56jyilHQ,https://www.youtube.com/watch?v=Y5G56jyilHQ&t=...
2,bir iki üç dört beş altı yedi sekiz,1781,1788,bak bir iki üç dört beş altı yedi sekiz dokuz ...,_wkFZF0ZuMU,https://www.youtube.com/watch?v=_wkFZF0ZuMU&t=...
3,bir iki üç dört beş altı yedi sekiz,1028,1034,bir iki üç dört beş altı yedi sekiz dokuz on,tQRsEpIj5uo,https://www.youtube.com/watch?v=tQRsEpIj5uo&t=...
4,bir iki üç dört beş altı yedi sekiz,3326,3333,bir iki üç dört beş altı yedi sekiz dokuz on,psNiD0Ks_uk,https://www.youtube.com/watch?v=psNiD0Ks_uk&t=...
...,...,...,...,...,...,...
73,hey hey hey hey hey hey hey hey,6455,6458,hey hey hey hey hey hey hey hey hey hey,_IimozO_x50,https://www.youtube.com/watch?v=_IimozO_x50&t=...
74,bu iyi bir şey mi kötü bir şey,4376,4379,acaba bu iyi bir şey mi kötü bir şey mi,KrLItbrSnZs,https://www.youtube.com/watch?v=KrLItbrSnZs&t=...
75,bu iyi bir şey mi kötü bir şey,4366,4369,bu iyi bir şey mi kötü bir şey mi,1mQL15DyOcM,https://www.youtube.com/watch?v=1mQL15DyOcM&t=...
76,dur dur dur dur dur dur dur dur,375,378,dur da ölmek çok gelecek şu anda biliyor musun...,OXmeI2243Ws,https://www.youtube.com/watch?v=OXmeI2243Ws&t=...


In [23]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,bir,13
1,şey,11
2,hayır,8
3,var,8
4,para,8
5,la,8
6,hey,8
7,dur,8
8,üç,7
9,dört,7


In [24]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link.xlsx", index=False)  ########*****

#### Copy Move And Delete

In [24]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_With_{word_end}_Word_Youtube_Link.xlsx")   ########*****
output_file

[]

In [137]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [138]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass